In [33]:
import pandas as pd
from twitter_bot_detection.helpers import log_running_time
from kedro.io import PickleLocalDataSet
from catboost.text_processing import Tokenizer

In [27]:
users = pd.read_pickle('data/03_primary/users.pkl')

In [32]:
users

,name,screen_name,location,description,protected,followers_count,friends_count,listed_count,created_at,favourites_count,...,statuses_count,profile_image_url_https,default_profile,default_profile_image,profile_banner_url,withheld_in_countries,label,last_status_date,profile_url,description_urls
id_str,,,,,,,,,,,,,,,,,,,,,
2718436417,Gruuvan,DJGruuvan,,"THE DJ GRUUVAN! You know who it is, baby.",False,103,110,0,2014-08-09 02:12:46+00:00,3,...,39,https://pbs.twimg.com/profile_images/497928954...,True,False,NaN,NaN,human,2018-04-24 16:23:41+00:00,http://ExchangeCorp.net,[]
792615528791703553,C C,Cc54518746C,,,False,36,102,10,2016-10-30 06:34:02+00:00,3701,...,6304,https://abs.twimg.com/sticky/default_profile_i...,True,True,NaN,NaN,human,2017-01-30 13:56:03+00:00,None,[]
3287012484,Penelope,penelope20mn,Heart & body H❤️ME in LA,Technically right · #BlessedAF · Engaged to th...,False,12949,4518,256,2015-07-22 02:53:07+00:00,103431,...,74237,https://pbs.twimg.com/profile_images/755098445...,True,False,https://pbs.twimg.com/profile_banners/32870124...,NaN,human,2020-02-19 05:01:43+00:00,https://twitter.com/search/from:penelope20mn%2...,[]
93816184,Soul Man,flipcritic,Australia,"Feminist, Polymath, Dilettante, Video Essayist...",False,2144,525,210,2009-12-01 08:05:46+00:00,111982,...,228753,https://pbs.twimg.com/profile_images/116732848...,False,False,https://pbs.twimg.com/profile_banners/93816184...,NaN,human,2020-02-20 14:04:53+00:00,http://moviemezzanine.com/author/mirasol/,[]
754884880996020225,Nils Hillestad,Trmpwin,New Zealand,#TREASON #PublicHangBest #HANG4 @HIllaryClinto...,False,4307,5417,92,2016-07-18 03:45:55+00:00,41375,...,103262,https://pbs.twimg.com/profile_images/810588272...,True,False,https://pbs.twimg.com/profile_banners/75488488...,NaN,bot,2017-02-27 07:43:44+00:00,http://twiends.com/trmpwin,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2734133342,Prince Sheri,sharoonp143,Gujranwala,,False,54,103,0,2014-08-15 10:10:57+00:00,157,...,946,https://pbs.twimg.com/profile_images/119854463...,False,False,https://pbs.twimg.com/profile_banners/27341333...,NaN,human,2019-12-02 09:51:29+00:00,https://www.youtube.com/channel/UCMoC3fw5RWsFg...,[]
160302029,Kelsey Cadden,kelseycadden,"Amory, MS | Oxford, MS","God is within her, she will not fail.",False,235,261,1,2010-06-27 20:08:42+00:00,3757,...,5158,https://pbs.twimg.com/profile_images/113785438...,False,False,https://pbs.twimg.com/profile_banners/16030202...,NaN,human,2020-02-07 05:49:49+00:00,None,[]
382361714,bhavesh,bhaveshc_,,,False,126,81,0,2011-09-29 23:06:51+00:00,11735,...,2911,https://pbs.twimg.com/profile_images/103084134...,False,False,NaN,NaN,human,2020-02-19 19:00:09+00:00,None,[]


In [43]:
def count_char_types(string):
    flags = [0, 0, 0, 0]
    for c in string:
        if c.islower():
            flags[0] = 1
        elif c.isupper():
            flags[1] = 1
        elif c.isdigit():
            flags[2] = 1
        else:
            flags[3] = 1
        
    return sum(flags)

In [46]:
@log_running_time
def extract_user_features(users: PickleLocalDataSet) -> PickleLocalDataSet:
    features = users[[
        "protected", "followers_count", "friends_count",  "listed_count", "favourites_count", "statuses_count",
        "verified", "default_profile", "default_profile_image",
    ]].copy()
    features["char_types"] = users["screen_name"].apply(count_char_types)
    features["has_location"] = users.location != ''
    features["has_description"] = users.description != ''
    features["created_at_time"] = pd.cut(users["created_at"].dt.hour, [-1, 6, 11, 18, 23], labels=["night", "morning", "day", "evening"])
    features["account_active_for_days"] = (users["last_status_date"] - users["created_at"]).dt.days
    features["has_banner"] = ~users["profile_banner_url"].isnull()
    features["has_profile_url"] = ~users["profile_url"].isnull()
    features["description_urls_count"] = users["description_urls"].str.len()
    features["label"] = (users["label"] == 'bot').astype(int)

    features["tweets_per_day"] = users["statuses_count"] / (features["account_active_for_days"] + 0.0000001)
    features["favourites_per_day"] = users["favourites_count"] / (features["account_active_for_days"] + 0.0000001)
    features["fr_to_flw_ratio"] = users["friends_count"] / (users["followers_count"] + 0.0000001)
    features["faw_to_tweets_ratio"] = users["favourites_count"] / (users["statuses_count"] + 0.0000001)
    features["tweets_to_faw_ratio"] = users["statuses_count"] / (users["favourites_count"] + 0.0000001)
    # features["listed_count_cat"] = pd.cut(users["listed_count"], [-1, 100, 500, 2000, 5000, 10000, float('inf')], labels=['100', '500', '2000', '5000', '10000', 'inf']).astype(str)

    return features.drop(columns=["protected"])

In [22]:
extract_user_features(users).to_pickle('data/04_features/user_features.pkl')

2020-03-10 23:09:28,816 - twitter_bot_detection.helpers - INFO - Running 'extract_user_features' took 0.04 seconds
